In [ ]:
gameFolder = "DOA++-MAME"

In [ ]:
import sys, os
import time
timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../'))   
sys.path.append(os.path.join(os.path.abspath(''), '../../../games',gameFolder))   

tensorBoardFolder = "./ppo2_TB_CustCnn/"
modelFolder = "ppo2_Model_CustCnn/"

os.makedirs(modelFolder, exist_ok=True)

In [ ]:
from makeDiambraEnv import *

import tensorflow as tf

from customPolicies.utils import linear_schedule, AutoSave
from customPolicies.customCnnPolicy import *

from stable_baselines import PPO2

In [ ]:
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../../roms/MAMEToolkit/roms/"
diambraKwargs["binary_path"] = "../../../../customMAME/"
diambraKwargs["player"] = "P1"
diambraKwargs["frame_ratio"] = 6
diambraKwargs["render"] = False
#diambraKwargs["throttle"] = False
#diambraKwargs["sound"] = False 
#diambraKwargs["character"] ="Random"
diambraKwargs["character"] = "Kasumi"

wrapperKwargs = {}
wrapperKwargs["hwc_obs_resize"] = [256, 256, 1]
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["clip_rewards"] = False
wrapperKwargs["frame_stack"] = 6
wrapperKwargs["scale"] = True
wrapperKwargs["scale_mod"] = 0


#keyToAdd = None
keyToAdd = []
keyToAdd.append("actionsBuf")
#keyToAdd.append("player")
keyToAdd.append("healthP1")
keyToAdd.append("healthP2")
keyToAdd.append("positionP1")
keyToAdd.append("positionP2")
#keyToAdd.append("winsP1")
#keyToAdd.append("winsP2")
keyToAdd.append("stage")

numEnv=8

env = make_diambra_env(diambraMame, env_prefix="Train", num_env=numEnv, seed=timeDepSeed, 
                       continue_game=0, diambra_kwargs=diambraKwargs, 
                       wrapper_kwargs=wrapperKwargs, key_to_add=keyToAdd)

In [ ]:
#print("Obs_space = ", env.observation_space)
#print("Obs_space type = ", env.observation_space.dtype)
#print("Obs_space high = ", env.observation_space.high)
#print("Obs_space low = ", env.observation_space.low)

In [ ]:
#print("Act_space = ", env.action_space)
#print("Act_space type = ", env.action_space.dtype)
#print("Act_space n = ", env.action_space.n)

In [ ]:
# Policy param
policyKwargs={}
policyKwargs["n_add_info"] = 149
policyKwargs["layers"] = [64, 64]

# PPO param
setGamma = 0.94
#setLearningRate = linear_schedule(2.5e-4, 2.5e-5)
#setClipRange = linear_schedule(0.15, 0.05)
setLearningRate = linear_schedule(1e-4, 2.5e-5)
setClipRange = linear_schedule(0.083, 0.05)
setClipRangeVf = setClipRange

# Initialize the model, 1 env
#model = PPO2(CustCnnPolicy, env, verbose=1, 
#             gamma = setGamma, nminibatches=4, noptepochs=4, n_steps=128,
#             learning_rate=setLearningRate, cliprange=setClipRange, cliprange_vf=setClipRangeVf, 
#             tensorboard_log=tensorBoardFolder, policy_kwargs=policyKwargs)

#OR

# Load the trained agent
model = PPO2.load(modelFolder+"13_5M", env=env, tensorboard_log=tensorBoardFolder, 
                  policy_kwargs=policyKwargs, gamma = setGamma, learning_rate=setLearningRate, 
                  cliprange=setClipRange, cliprange_vf=setClipRangeVf)

In [ ]:
print("Model discount factor = ", model.gamma)

In [ ]:
# Create the callback: autosave every USER DEF steps
autoSaveCallback = AutoSave(check_freq=1000000, numEnv=numEnv, save_path=modelFolder+"13_5M_")

# Train the agent
time_steps = 20000000
model.learn(total_timesteps=time_steps, callback=autoSaveCallback)

In [ ]:
# Save the agent
model.save(modelFolder+"33_5M")

## Evaluation

In [ ]:
# Initialize a new evaluation environment
diambraKwargs["render"] = True

env = make_diambra_env(diambraMame, env_prefix="Eval", num_env=1, seed=timeDepSeed, 
                       continue_game=0, diambra_kwargs=diambraKwargs, 
                       wrapper_kwargs=wrapperKwargs, key_to_add=keyToAdd)

In [ ]:
observation = env.reset()

cumulativeEpRew = 0.0
cumulativeEpRewAll = []
cumulativeTotRew = 0.0

maxNumEp = 100
currNumEp = 0

while currNumEp < maxNumEp:

    action = model.predict(observation, deterministic=False)
    #action_prob = model.action_probability(observation, states)
    #print("Action probabilities = ", action_prob)
    #print("Max action = ", np.argmax(action_prob))
    #print("Action = ", action)
    
    observation, reward, done, info = env.step(action[0])
    
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeTotRew += cumulativeEpRew
        cumulativeEpRew = 0.0

print("Mean cumulative reward = ", cumulativeTotRew/maxNumEp)    
print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))   
    
env.close()